# Update OSS account_number to Match SF Name

Updates `customers.accounts.account_number` in OSS to match the `Name` field from SF `billing_invoice__c`.

**IMPORTANT:** Disable `set_account_number_trig` before running live:
```sql
ALTER TABLE customers.accounts DISABLE TRIGGER set_account_number_trig;
```

**After running, re-enable:**
```sql
ALTER TABLE customers.accounts ENABLE TRIGGER set_account_number_trig;
```

**Matching Logic:**
- OSS `account_id` = SF `account_number__c`
- Only where SF `bbf_ban__c = true` AND OSS `bbf_ban = true`

**Safety:**
- `DRY_RUN = True` by default

In [10]:
# === SETUP ===
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from datetime import datetime

# Connection credentials
heroku_conn_kwargs = {
    "dbname": "d88otjf7uhv9pr",
    "user": "ucn7cbk14sd6h",
    "password": "pf27d102f95e996e621e02523d035a1bff27590c8e6a13f5b180703a6631320c5",
    "host": "ec2-54-86-217-174.compute-1.amazonaws.com",
    "port": "5432",
    "connect_timeout": 10,
}

oss_conn_kwargs = {
    "dbname": "GLC",
    "user": "oss_server",
    "password": "3wU3uB28X?!r2?@ebrUg",
    "host": "pg01.comlink.net",
    "port": "5432",
    "connect_timeout": 10,
}

print("Connecting to databases...")
conn = psycopg2.connect(**heroku_conn_kwargs, cursor_factory=RealDictCursor)
print("✅ Connected to Heroku")
oconn = psycopg2.connect(**oss_conn_kwargs, cursor_factory=RealDictCursor)
print("✅ Connected to OSS")

Connecting to databases...
✅ Connected to Heroku
✅ Connected to OSS


In [11]:
# === CONFIGURATION ===
DRY_RUN = False  # Set to False to actually update records

print(f"DRY_RUN: {DRY_RUN}")
if not DRY_RUN:
    print("\n⚠️  WARNING: LIVE RUN - Records will be updated!")
    print("⚠️  Make sure set_account_number_trig is DISABLED before proceeding!")

DRY_RUN: False

⚠️  WARNING: LIVE RUN - Records will be updated!
⚠️  Make sure set_account_number_trig is DISABLED before proceeding!


In [12]:
# === STEP 1: Get SF BBF BANs ===

sf_bbf_bans_sql = """
SELECT 
    account_number__c,
    name,
    sfid
FROM sfprod.billing_invoice__c
WHERE bbf_ban__c = true
  AND account_number__c IS NOT NULL
"""

print("Fetching SF BBF BANs...")
with conn.cursor() as cur:
    cur.execute(sf_bbf_bans_sql)
    sf_bbf_bans = cur.fetchall()

sf_bbf_df = pd.DataFrame(sf_bbf_bans)
print(f"✅ Found {len(sf_bbf_df)} SF BBF BANs")
print(sf_bbf_df.head(10))

Fetching SF BBF BANs...
✅ Found 2505 SF BBF BANs
  account_number__c         name                sfid
0            117702  A117702-BBF  aA3Qp000000AFiXKAW
1            118278  A118278-BBF  aA3Qp000000ASELKA4
2            117703  A117703-BBF  aA3Qp000000AFk9KAG
3            117704  A117704-BBF  aA3Qp000000AFllKAG
4            117705  A117705-BBF  aA3Qp000000AFnNKAW
5            117706  A117706-BBF  aA3Qp000000AFozKAG
6            117707  A117707-BBF  aA3Qp000000AFqbKAG
7            117708  A117708-BBF  aA3Qp000000AE9mKAG
8            117710  A117710-BBF  aA3Qp000000AFtpKAG
9            118279  A118279-BBF  aA3Qp000000ASFxKAO


In [13]:
# === STEP 2: Get OSS BBF accounts ===

oss_bbf_accounts_sql = """
SELECT 
    account_id,
    account_number,
    account_nm,
    bbf_ban
FROM customers.accounts
WHERE bbf_ban = true
"""

print("Fetching OSS BBF accounts...")
with oconn.cursor() as ocur:
    ocur.execute(oss_bbf_accounts_sql)
    oss_bbf_accounts = ocur.fetchall()

oss_bbf_df = pd.DataFrame(oss_bbf_accounts)
print(f"✅ Found {len(oss_bbf_df)} OSS BBF accounts")
print(oss_bbf_df.head(10))

Fetching OSS BBF accounts...
✅ Found 2505 OSS BBF accounts
   account_id  account_number                         account_nm  bbf_ban
0      117308  A91910264992-W  Upper Peninsula Telephone Company     True
1      117309  A91910264901-R         Holland Community Hospital     True
2      117310  A91910264018-R        Charter Township of Clinton     True
3      117311  A91910264026-R            Rockford Public Schools     True
4      117312  A91910264034-R                       Sunset Manor     True
5      119258  E91910283497-R                        Turner Ohio     True
6      117313  A91910264042-W                  Stellar Broadband     True
7      117314  A91910264059-W                WOW! Wide Open West     True
8      117315  A91910264067-R                         Invisalink     True
9      117316  A91910264075-R                               KPEP     True


In [14]:
# === STEP 3: Match and identify updates needed ===

# Create lookup: SF account_number__c -> SF name
sf_lookup = {row["account_number__c"]: row["name"] for row in sf_bbf_bans}

updates_needed = []
already_correct = []
no_sf_match = []

for oss_row in oss_bbf_accounts:
    oss_account_id = str(oss_row["account_id"])
    oss_account_number = oss_row["account_number"]

    # Find matching SF record
    sf_name = sf_lookup.get(oss_account_id)

    if sf_name is None:
        no_sf_match.append(
            {
                "account_id": oss_row["account_id"],
                "current_account_number": oss_account_number,
                "account_nm": oss_row["account_nm"],
            }
        )
    elif oss_account_number == sf_name:
        already_correct.append(
            {
                "account_id": oss_row["account_id"],
                "account_number": oss_account_number,
                "account_nm": oss_row["account_nm"],
            }
        )
    else:
        updates_needed.append(
            {
                "account_id": oss_row["account_id"],
                "current_account_number": oss_account_number,
                "new_account_number": sf_name,
                "account_nm": oss_row["account_nm"],
            }
        )

print(f"\n=== Analysis ===")
print(f"  Already correct: {len(already_correct)}")
print(f"  Updates needed: {len(updates_needed)}")
print(f"  No SF match: {len(no_sf_match)}")


=== Analysis ===
  Already correct: 0
  Updates needed: 2505
  No SF match: 0


In [15]:
# === STEP 4: Preview updates ===

print("\n=== Updates Preview (first 20) ===")
updates_df = pd.DataFrame(updates_needed)
if len(updates_df) > 0:
    print(updates_df.head(20).to_string())
else:
    print("No updates needed!")


=== Updates Preview (first 20) ===
    account_id current_account_number new_account_number                         account_nm
0       117308         A91910264992-W        A117308-BBF  Upper Peninsula Telephone Company
1       117309         A91910264901-R        A117309-BBF         Holland Community Hospital
2       117310         A91910264018-R        A117310-BBF        Charter Township of Clinton
3       117311         A91910264026-R        A117311-BBF            Rockford Public Schools
4       117312         A91910264034-R        A117312-BBF                       Sunset Manor
5       119258         E91910283497-R        A119258-BBF                        Turner Ohio
6       117313         A91910264042-W        A117313-BBF                  Stellar Broadband
7       117314         A91910264059-W        A117314-BBF                WOW! Wide Open West
8       117315         A91910264067-R        A117315-BBF                         Invisalink
9       117316         A91910264075-R       

In [16]:
# === STEP 5: Apply updates ===
# Sets account_number = SF Name value
# REMINDER: Trigger must be disabled for this to work

results = []
success_count = 0
error_count = 0

print(f"\n{'DRY RUN - ' if DRY_RUN else ''}Processing {len(updates_needed)} updates...")

for idx, update in enumerate(updates_needed, 1):
    account_id = update["account_id"]
    new_account_number = update["new_account_number"]

    result = {
        "account_id": account_id,
        "current_account_number": update["current_account_number"],
        "new_account_number": new_account_number,
        "account_nm": update["account_nm"],
        "status": None,
        "error": None,
    }

    if DRY_RUN:
        result["status"] = "WOULD_UPDATE"
        success_count += 1
    else:
        try:
            with oconn.cursor() as ocur:
                ocur.execute(
                    """
                    UPDATE customers.accounts
                    SET account_number = %s
                    WHERE account_id = %s
                      AND bbf_ban = true
                """,
                    (new_account_number, account_id),
                )
            oconn.commit()
            result["status"] = "UPDATED"
            success_count += 1

            if idx % 100 == 0:
                print(f"  Processed {idx}/{len(updates_needed)}...")

        except Exception as e:
            oconn.rollback()
            result["status"] = "ERROR"
            result["error"] = str(e)
            error_count += 1
            print(f"  ❌ Error on account_id {account_id}: {e}")

    results.append(result)

print(f"\n=== Summary ===")
print(f"  {'Would update' if DRY_RUN else 'Updated'}: {success_count}")
print(f"  Errors: {error_count}")


Processing 2505 updates...
  Processed 100/2505...
  Processed 200/2505...
  Processed 300/2505...
  Processed 400/2505...
  Processed 500/2505...
  Processed 600/2505...
  Processed 700/2505...
  Processed 800/2505...
  Processed 900/2505...
  Processed 1000/2505...
  Processed 1100/2505...
  Processed 1200/2505...
  Processed 1300/2505...
  Processed 1400/2505...
  Processed 1500/2505...
  Processed 1600/2505...
  Processed 1700/2505...
  Processed 1800/2505...
  Processed 1900/2505...
  Processed 2000/2505...
  Processed 2100/2505...
  Processed 2200/2505...
  Processed 2300/2505...
  Processed 2400/2505...
  Processed 2500/2505...

=== Summary ===
  Updated: 2505
  Errors: 0


In [17]:
# === STEP 6: Save results ===

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"oss_account_number_update_{run_type}_{timestamp}.xlsx"

from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill

wb = Workbook()

# Sheet 1: Summary
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["OSS account_number Update Summary"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append([])
ws1.append(["Already correct:", len(already_correct)])
ws1.append(["Updates needed:", len(updates_needed)])
ws1.append(["No SF match:", len(no_sf_match)])
ws1.append([])
ws1.append(["Success:", success_count])
ws1.append(["Errors:", error_count])

# Sheet 2: Updates
ws2 = wb.create_sheet("Updates")
ws2.append(
    [
        "account_id",
        "account_nm",
        "current_account_number",
        "new_account_number",
        "status",
        "error",
    ]
)
for r in results:
    ws2.append(
        [
            r["account_id"],
            r["account_nm"],
            r["current_account_number"],
            r["new_account_number"],
            r["status"],
            r["error"],
        ]
    )

# Sheet 3: Already Correct
ws3 = wb.create_sheet("Already Correct")
ws3.append(["account_id", "account_nm", "account_number"])
for r in already_correct:
    ws3.append([r["account_id"], r["account_nm"], r["account_number"]])

# Sheet 4: No SF Match
ws4 = wb.create_sheet("No SF Match")
ws4.append(["account_id", "account_nm", "current_account_number"])
for r in no_sf_match:
    ws4.append([r["account_id"], r["account_nm"], r["current_account_number"]])

wb.save(output_file)
print(f"\n✅ Results saved to: {output_file}")


✅ Results saved to: oss_account_number_update_live_run_20260109_103445.xlsx


In [18]:
# === Cleanup ===
conn.close()
oconn.close()
print("🔌 Database connections closed.")

if DRY_RUN:
    print("\n🔄 DRY RUN complete. No records were updated.")
    print("\nTo run for real:")
    print(
        "1. Disable trigger: ALTER TABLE customers.accounts DISABLE TRIGGER set_account_number_trig;"
    )
    print("2. Set DRY_RUN = False and re-run")
    print(
        "3. Re-enable trigger: ALTER TABLE customers.accounts ENABLE TRIGGER set_account_number_trig;"
    )
else:
    print(f"\n✅ LIVE RUN complete. {success_count} records updated.")
    print("\n⚠️  Don't forget to re-enable the trigger:")
    print("ALTER TABLE customers.accounts ENABLE TRIGGER set_account_number_trig;")

🔌 Database connections closed.

✅ LIVE RUN complete. 2505 records updated.

⚠️  Don't forget to re-enable the trigger:
ALTER TABLE customers.accounts ENABLE TRIGGER set_account_number_trig;
